In [31]:
# requirements
from pymongo import MongoClient

import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA

import joblib

import tqdm
import heapq
import json

In [37]:
# MongoDB 연결
client = MongoClient("mongodb://localhost:3999/")

In [38]:
# DB 불러오기
matchup_statistics_db = client["matchup_statistics_db"]  # summoners, indicators, matches

In [36]:
for collection_name in matchup_statistics_db.list_collection_names():
    print(collection_name)
    collection = matchup_statistics_db["indicators"]

    documents = collection.find()
    cnt = 0

    for document in tqdm.tqdm(documents):
        print(document)
        break
        cnt += 1

    print(cnt)

summoners


0it [00:00, ?it/s]


{'_id': 't3C2JzNt9v8ypQ3PtL7v4c6VNvn-QdDDnm8-Pcnllyd65xPT', 'matchIndicators': [{'matchId': 'KR_6992198203', 'metadata': {'laneInfo': {'teamPosition': 'BOTTOM', 'isBottomLane': True, 'myTeamId': 100, 'myLaneNumber': 4, 'oppositeLaneNumber': 9, 'myBottomDuoNumber': 5, 'oppositeBottomDuoNumber': 10}, 'timeInfo': {'gameDuration': 1663, 'startTime': 1710722466095, 'endTime': 1710724129616}, 'isFinishedBeforeFifteen': False, 'isOurTeamEarlySurrendered': False, 'isWin': False, 'pingCount': 0, 'champion': 'Smolder'}, 'laneIndicator': {'basicWeight': {'csDiffer': 13, 'expDiffer': 760, 'turretPlateDestroyDiffer': 3.0, 'supportItemFinishedTimeDiffer': 0}, 'aggresiveLaneAbilility': {'dealDiffer': -4025, 'soloKillDiffer': 0, 'duoKillDiffer': -2}, '_class': 'com.ssafy.matchup_statistics.indicator.entity.match.beginning.BtmIndicator'}, 'macroIndicator': {'splitPoint': {'turretKillsPerDeaths': 0, 'damageDealtToTurretsPerTotalDamageDealt': 430, 'damageDealtToTurretsPerTeamTotalTowerDamageDone': 24838}

0it [00:00, ?it/s]

{'_id': 't3C2JzNt9v8ypQ3PtL7v4c6VNvn-QdDDnm8-Pcnllyd65xPT', 'matchIndicators': [{'matchId': 'KR_6992198203', 'metadata': {'laneInfo': {'teamPosition': 'BOTTOM', 'isBottomLane': True, 'myTeamId': 100, 'myLaneNumber': 4, 'oppositeLaneNumber': 9, 'myBottomDuoNumber': 5, 'oppositeBottomDuoNumber': 10}, 'timeInfo': {'gameDuration': 1663, 'startTime': 1710722466095, 'endTime': 1710724129616}, 'isFinishedBeforeFifteen': False, 'isOurTeamEarlySurrendered': False, 'isWin': False, 'pingCount': 0, 'champion': 'Smolder'}, 'laneIndicator': {'basicWeight': {'csDiffer': 13, 'expDiffer': 760, 'turretPlateDestroyDiffer': 3.0, 'supportItemFinishedTimeDiffer': 0}, 'aggresiveLaneAbilility': {'dealDiffer': -4025, 'soloKillDiffer': 0, 'duoKillDiffer': -2}, '_class': 'com.ssafy.matchup_statistics.indicator.entity.match.beginning.BtmIndicator'}, 'macroIndicator': {'splitPoint': {'turretKillsPerDeaths': 0, 'damageDealtToTurretsPerTotalDamageDealt': 430, 'damageDealtToTurretsPerTeamTotalTowerDamageDone': 24838}

...


0
matches


0it [00:00, ?it/s]

{'_id': 't3C2JzNt9v8ypQ3PtL7v4c6VNvn-QdDDnm8-Pcnllyd65xPT', 'matchIndicators': [{'matchId': 'KR_6992198203', 'metadata': {'laneInfo': {'teamPosition': 'BOTTOM', 'isBottomLane': True, 'myTeamId': 100, 'myLaneNumber': 4, 'oppositeLaneNumber': 9, 'myBottomDuoNumber': 5, 'oppositeBottomDuoNumber': 10}, 'timeInfo': {'gameDuration': 1663, 'startTime': 1710722466095, 'endTime': 1710724129616}, 'isFinishedBeforeFifteen': False, 'isOurTeamEarlySurrendered': False, 'isWin': False, 'pingCount': 0, 'champion': 'Smolder'}, 'laneIndicator': {'basicWeight': {'csDiffer': 13, 'expDiffer': 760, 'turretPlateDestroyDiffer': 3.0, 'supportItemFinishedTimeDiffer': 0}, 'aggresiveLaneAbilility': {'dealDiffer': -4025, 'soloKillDiffer': 0, 'duoKillDiffer': -2}, '_class': 'com.ssafy.matchup_statistics.indicator.entity.match.beginning.BtmIndicator'}, 'macroIndicator': {'splitPoint': {'turretKillsPerDeaths': 0, 'damageDealtToTurretsPerTotalDamageDealt': 430, 'damageDealtToTurretsPerTeamTotalTowerDamageDone': 24838}

0it [00:00, ?it/s]


0


In [40]:
iron_users = []

for summoner in matchup_statistics_db["summoners"].find():
    if summoner["league"]["tier"] == "SILVER" or summoner["league"]["tier"] == "BRONZE" or summoner["league"]["tier"] == "IRON":
        iron_users.append(summoner["_id"])

print(len(iron_users))
print(iron_users[0])

13212
Ar_KKTRQ4khe-Lm8iDZzy5GLzag_4QfvTCZyXkQYodvTVBmx


In [41]:
# 학습 데이터 추출

# collection 불러오기
indicators = matchup_statistics_db["indicators"].find()

# 학습 데이터
x = []
y = []

summoners = []

times = 0
all_zero = 0


# 결측치 처리 - 추후에는 평균 값으로 입력
def imputation(datas, indicator, record, record_detail):
    global times

    if datas[indicator][record].get(record_detail):
        return datas[indicator][record].get(record_detail)
    else:
        times += 1
        return 0

# 데이터 추출
for indicator in indicators:
    if indicator["_id"] not in iron_users:
        continue

    for match_indicator in indicator["matchIndicators"]:
        summoner = {}

        summoner["csDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["csDiffer"]
        summoner["expDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["expDiffer"]
        summoner["turretPlateDestroyDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["turretPlateDestroyDiffer"]
        summoner["dealDiffer"] = imputation(match_indicator, "laneIndicator", "aggresiveLaneAbilility", "dealDiffer")

        summoner["turretKillsPerDeaths"] = match_indicator["macroIndicator"]["splitPoint"]["turretKillsPerDeaths"]
        summoner["damageDealtToTurretsPerTeamTotalTowerDamageDone"] = match_indicator["macroIndicator"]["splitPoint"]["damageDealtToTurretsPerTeamTotalTowerDamageDone"]

        summoner["totalTimeCCingOthersPerTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["totalTimeCCingOthersPerTotalDamageTaken"]
        summoner["totalDamageTakenPerTeamTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["totalDamageTakenPerTeamTotalDamageTaken"]
        summoner["damageSelfMitigatedPerTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["damageSelfMitigatedPerTotalDamageTaken"]

        summoner["visionScorePerDeath"] = match_indicator["macroIndicator"]["visionPoint"]["visionScorePerDeath"]
        summoner["totalJungleObjectivePerGameDuration"] = match_indicator["macroIndicator"]["jungleHoldPoint"]["totalJungleObjectivePerGameDuration"]

        summoner["getObjectiveDifferPerGameDuration"] = match_indicator["macroIndicator"]["objectivePoint"]["getObjectiveDifferPerGameDuration"]

        summoner["damagePerMinute"] = match_indicator["macroIndicator"]["totalDealPoint"]["damagePerMinute"]
        summoner["dealPerGold"] = imputation(match_indicator, "macroIndicator", "totalDealPoint", "dealPerGold")
        summoner["teamDamagePercentage"] = imputation(match_indicator, "macroIndicator", "totalDealPoint", "teamDamagePercentage")

        if match_indicator["metadata"]["laneInfo"].get("teamPosition") == "TOP":
            pass


        is_all_zero = True
        idx = 0

        for key, value in summoner.items():
            if value != 0:
                is_all_zero = False
                break 
                
        if is_all_zero is False:

            if match_indicator["metadata"]["isWin"] is True:
                y.append(1)
            else:
                y.append(0)

            summoners.append(summoner)

In [11]:
df = pd.DataFrame(summoners)
df

,csDiffer,expDiffer,turretPlateDestroyDiffer,dealDiffer,turretKillsPerDeaths,damageDealtToTurretsPerTeamTotalTowerDamageDone,totalTimeCCingOthersPerTotalDamageTaken,totalDamageTakenPerTeamTotalDamageTaken,damageSelfMitigatedPerTotalDamageTaken,visionScorePerDeath,totalJungleObjectivePerGameDuration,getObjectiveDifferPerGameDuration,damagePerMinute,dealPerGold,teamDamagePercentage
0,-4,56,-1.000000,-1872,0,25073,102,7969,42696,600000,0,194,9300000,28147,0.047714
1,-13,-868,0.000000,-3674,14285,14861,92,13384,51275,442857,0,-346,29300000,96640,0.087105
2,-67,-1586,4.000000,-271,0,9867,21,22273,78201,133333,0,61,49900000,186560,0.173014
3,3,-1121,0.000000,-895,0,0,204,16185,46230,354545,0,198,50600000,187540,0.178610
4,16,1763,0.000000,3183,0,15128,197,21713,55998,418181,0,-296,61700000,191331,0.194767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241569,70,1770,-5.000000,1767,83333,42047,113,19221,123841,450000,673,504,111200000,198174,0.284319
241570,18,1193,1.333333,2455,20000,18660,86,28845,114744,680000,563,46,47100000,118424,0.181526
241571,26,2291,-2.000000,4351,0,13585,106,26460,121087,533333,731,121,74600000,148871,0.169784
241572,7,-694,2.000000,-2673,0,12921,0,17350,50990,155555,0,-163,61000000,165384,0.236524


In [12]:
# PCA
df_normalized = (df - df.min()) / (df.max() - df.min())

In [13]:
scaler = StandardScaler()
x = df_normalized.to_numpy()
x_scaled = scaler.fit_transform(x)

In [14]:
print(x[0])

[0.53380783 0.5253067  0.4        0.50333346 0.         0.25074504
 0.0009     0.13922059 0.02653357 0.04651163 0.         0.60220994
 0.02926369 0.04258284 0.07139737]


In [15]:
parameter_space = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [16]:
# mlp = MLPClassifier(hidden_layer_sizes=(75, ), max_iter=500, activation='relu', solver='adam', random_state=42, alpha=0.00001, verbose=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
mlp = MLPClassifier(max_iter=500)
random_search = RandomizedSearchCV(mlp, param_distributions=parameter_space, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)
random_search.fit(x_train, y_train)
#mlp.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=MLPClassifier(max_iter=500), n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'alpha': [0.0001, 0.05],
                                        'hidden_layer_sizes': [(50,), (100,),
                                                               (50, 50),
                                                               (100, 50)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'solver': ['sgd', 'adam']},
                   random_state=42, verbose=2)

In [17]:
print("최적의 하이퍼파라미터:", random_search.best_params_)
print("최고 점수:", random_search.best_score_)

최적의 하이퍼파라미터: {'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (100,), 'alpha': 0.0001, 'activation': 'tanh'}
최고 점수: 0.8688754608747429


In [18]:
mlp = MLPClassifier(hidden_layer_sizes=(100, ), max_iter=500, activation='tanh', solver='adam', random_state=42, alpha = 0.0001, learning_rate = 'constant', verbose=True)
mlp.fit(x_train, y_train)

Iteration 1, loss = 0.42530298
Iteration 2, loss = 0.35035344
Iteration 3, loss = 0.34478852
Iteration 4, loss = 0.34404125
Iteration 5, loss = 0.34269821
Iteration 6, loss = 0.34204876
Iteration 7, loss = 0.34166299
Iteration 8, loss = 0.33992045
Iteration 9, loss = 0.33764969
Iteration 10, loss = 0.33500532
Iteration 11, loss = 0.33267854
Iteration 12, loss = 0.33103678
Iteration 13, loss = 0.32977164
Iteration 14, loss = 0.32866925
Iteration 15, loss = 0.32804543
Iteration 16, loss = 0.32745096
Iteration 17, loss = 0.32715966
Iteration 18, loss = 0.32660355
Iteration 19, loss = 0.32620820
Iteration 20, loss = 0.32563544
Iteration 21, loss = 0.32509709
Iteration 22, loss = 0.32521383
Iteration 23, loss = 0.32434838
Iteration 24, loss = 0.32385886
Iteration 25, loss = 0.32330201
Iteration 26, loss = 0.32295714
Iteration 27, loss = 0.32259506
Iteration 28, loss = 0.32211398
Iteration 29, loss = 0.32121152
Iteration 30, loss = 0.32121460
Iteration 31, loss = 0.32100017
Iteration 32, los

MLPClassifier(activation='tanh', max_iter=500, random_state=42, verbose=True)

In [21]:
joblib.dump(mlp, 'models/winning/bronze_model.pkl')

['models/winning/bronze_model.pkl']

In [99]:
max_heapq = []

probabilities = mlp.predict_proba(x_test)
cnt = 0
total = 0
for i, probability in enumerate(probabilities):
    heapq.heappush(max_heapq, (-probability[1], i))
    total += 1
    if probability[1] > 0.9:
        cnt += 1
    # print(f"{i}번째 승리확률 : {probability[1]}")

print(total)
print(cnt)

for _ in range(5):
    print(heapq.heappop(max_heapq))

[[0.5480427  0.62149135 0.5        ... 0.13750787 0.296798   0.25519632]
 [0.59786477 0.60837823 0.3        ... 0.27092511 0.38785193 0.63389444]
 [0.51601423 0.52211511 0.5        ... 0.14317181 0.24541629 0.22681683]
 ...
 [0.61209964 0.64615821 0.1        ... 0.34833228 0.40505777 0.61567914]
 [0.59074733 0.57293734 0.5        ... 0.13624921 0.25820276 0.2334923 ]
 [0.49466192 0.49907854 0.6        ... 0.23348018 0.29540348 0.39018069]]
48551
12356
(-0.9999999839766744, 19896)
(-0.9999999779354829, 7970)
(-0.9999999652883452, 8818)
(-0.9999999592109347, 46684)
(-0.9999999435980574, 44015)
